# Compare the two versions of interpretation of the formula

### Some functions to prepare, including the function to calculate P(si) and P(sj|si)

In [1]:
import os
import itertools
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr

# create the dict of key: seq, value: count
def generate_dict_count(fn_outcomes):
    dict_count = {}
    with open(fn_outcomes, "r") as fp:
        for line in fp:
            parts = line.rstrip().split("\t")
            seq = parts[0]
            count = int(parts[1])
            dict_count[seq] = count

    return dict_count

# generate only theoritical bystander outcomes (original target seq + only C->T within editing window)
def generate_bystander_outcomes_expect(seq, start, end):
    index_start = int(start) - 1
    index_end = int(end) -1
    outcomes = ['']
    for i in range(len(seq)):
        if index_start <= i <= index_end:
            new_outcomes = []
            for outcome in outcomes:
                if seq[i] == "C":
                    new_outcomes.append(outcome + seq[i])
                    new_outcomes.append(outcome + 'T')
                else:
                    new_outcomes.append(outcome + seq[i])
            outcomes = new_outcomes
        else:
            outcomes = [outcome + seq[i] for outcome in outcomes]
    return outcomes

# construct the occupancies of 4 nucleotides at each position of one target sequence (together with its editing outcome)
def prepare_probs(fn_outcomes):
    # Extract the sequence from the fn
    target_seq = os.path.basename(fn_outcomes).split("_")[0]
    # occupancy_list = [{"A":0,"T":0,"G":0,"C":0} for x in range(len(target_seq))]

    dict_edit_freq = {}
    for index in range(len(target_seq)):
        for nucl in ["C", "Non-C"]:
            dict_edit_freq[(index, nucl)] = {'A': 0, 'C': 0, 'G': 0, 'T': 0}

    dict_coedit_freq = {}  # co-editing frequency
    # for i, j in itertools.product(range(start-1, end), range(start-1, end)):
    #     for status_i, status_j in itertools.product("C-C", "C-T"):
    #         dict_coedit_freq[(i, j), (status_i, status_j)] = 0

    for i, j in itertools.product(range(len(target_seq)), range(len(target_seq))):
        for m,n in itertools.product(['A', 'C', 'G', 'T'], ['A', 'C', 'G', 'T']):
            dict_coedit_freq[(i, j), (m, n)] = {
                'A': {'A': 0, 'C': 0, 'G': 0, 'T': 0},
                'C': {'A': 0, 'C': 0, 'G': 0, 'T': 0},
                'G': {'A': 0, 'C': 0, 'G': 0, 'T': 0},
                'T': {'A': 0, 'C': 0, 'G': 0, 'T': 0}
            }
        
    dict_count = generate_dict_count(fn_outcomes)

    for outcome in dict_count.keys():
        # build the editing probability matrix
        for index in range(len(target_seq)):
            t = target_seq[index]
            e = outcome[index]  # t: nucleotide of target seq; e: nucleotide of editing outcome
            # occupancy_list[index][e] += dict_count[outcome]
            if t == 'C':
                dict_edit_freq[(index, 'C')][e] += dict_count[outcome]
            else:
                dict_edit_freq[(index, 'Non-C')][e] += dict_count[outcome]
        # build the editing conditional probability matrix
        for i, j in itertools.product(range(len(target_seq)), range(len(target_seq))):
            # the conditional probability is P(S_j | S_i)
            t_i = target_seq[i]
            e_i = outcome[i]
            t_j = target_seq[j]
            e_j = outcome[j]

            dict_coedit_freq[(i, j), (t_i, e_i)][t_j][e_j] += dict_count[outcome]
    
    dict_edit_rate = {}
    for index in range(len(target_seq)):
        if sum(dict_edit_freq[(index, 'C')].values()) > 0:
            dict_edit_rate[(index, "edited")] = dict_edit_freq[(index, 'C')]['T'] / sum(dict_edit_freq[(index, 'C')].values())
            dict_edit_rate[(index, "not-edited")] = dict_edit_freq[(index, 'C')]['C'] / sum(dict_edit_freq[(index, 'C')].values())
        else:
            dict_edit_rate[(index, "edited")] = 0
            dict_edit_rate[(index, "not-edited")] = 0

    dict_coedit_rate = {}
    for i, j in itertools.product(range(len(target_seq)), range(len(target_seq))):
        dict_coedit_rate[(i,j), ('edited', 'edited')] = dict_coedit_freq[(i, j), ('C','T')]['C']['T']/dict_edit_freq[(i, 'C')]['T'] if dict_edit_freq[(i, 'C')]['T'] > 0 else 1
        dict_coedit_rate[(i,j), ('edited', 'not-edited')] = dict_coedit_freq[(i, j), ('C','T')]['C']['C']/dict_edit_freq[(i, 'C')]['T'] if dict_edit_freq[(i, 'C')]['T'] > 0  else 1
        dict_coedit_rate[(i,j), ('not-edited', 'edited')] = dict_coedit_freq[(i, j), ('C','C')]['C']['T']/dict_edit_freq[(i, 'C')]['C'] if dict_edit_freq[(i, 'C')]['C'] > 0 else 1
        dict_coedit_rate[(i,j), ('not-edited', 'not-edited')] = dict_coedit_freq[(i, j), ('C','C')]['C']['C']/dict_edit_freq[(i, 'C')]['C'] if dict_edit_freq[(i, 'C')]['C'] > 0 else 1
    
    return target_seq, dict_edit_freq, dict_edit_rate, dict_coedit_freq, dict_coedit_rate

### Interpretation version 1: the elements in E are the positions that the C is edited to T from target sequence to a particular bystander (within editing window)

In [2]:
def calc_prob_bystanders_v1(target_seq, start, end, dict_edit_rate, dict_coedit_rate):
    index_start = int(start) - 1
    index_end = int(end) -1

    bystanders = generate_bystander_outcomes_expect(target_seq, start, end)

    dict_bystanders_prob = {}
    
    for bystander in bystanders:
        if bystander == target_seq:
            continue
            
        E = [index for index, (char1, char2) in enumerate(zip(target_seq, bystander)) if char1 != char2]
        
        value = 1
        for i in E:
            P_i = dict_edit_rate[(i, 'edited')] if (i, 'edited') in dict_edit_rate.keys() else 0
            value = value * P_i
            for j in E:
                if j == i:
                    pass
                else:
                    P_ij = dict_coedit_rate[((i,j), ('edited', 'edited'))] if ((i,j), ('edited', 'edited')) in dict_coedit_rate.keys() else 1
                    value = value * P_ij
                    
        value = value**(1.0/len(E))

        dict_bystanders_prob[bystander] = value

    return dict_bystanders_prob

### Interpretation version 2: the elements in E are the positions of C in target sequence (regardless of bystander outcomes, within editing window)

In [3]:
def calc_prob_bystanders_v2(target_seq, start, end, dict_edit_rate, dict_coedit_rate):
    index_start = start - 1
    index_end = end -1

    bystanders = generate_bystander_outcomes_expect(target_seq, start, end)

    dict_bystanders_prob = {}

    # the indexes of "C" in target_seq 
    E = [i for i in range(len(target_seq)) if index_start <= i <= index_end and target_seq[i] == "C"]
    
    for bystander in bystanders:
        if bystander == target_seq:
            continue
        
        value = 1
        for i in E:
            if bystander[i] == "T":
                edit_status = 'edited'
            else:
                edit_status = 'not-edited'
            
            P_i = dict_edit_rate.get((i, edit_status), 0)
            value *= P_i
            
            for j in E:
                if j != i:
                    if bystander[j] == "T":
                        coedit_status = 'edited'
                    else:
                        coedit_status = 'not-edited'
                    
                    P_ij = dict_coedit_rate.get(((i,j), (edit_status, coedit_status)), 1)
                    value *= P_ij
                    
        value = value**(1.0/len(E))

        dict_bystanders_prob[bystander] = value

    return dict_bystanders_prob

### The read count of all editing outcomes (including the original target sequence and C->T outside of editing window)

In [4]:
dict_count = generate_dict_count('TGTGTCGTTTTCGGTTGTCTG_outcomes.tsv')
dict_count

{'TGTGTCGTTTTCGGTTGTCTG': 7869,
 'TGTGTTGTTTTTGGTTGTCTG': 993,
 'TGTGTTGTTTTCGGTTGTCTG': 144,
 'TGTGTCGTTTTTGGTTGTCTG': 33,
 'TGTGTCGTTTTCGGTTGTTTG': 18,
 'TGTGTTGTTTTTGGTTGTTTG': 8,
 'TGTGTCGTTTTTGGTTGTTTG': 2,
 'TGTGTTGTTTTCGGTTGTTTG': 1}

### Prepare the probabilities and conditional probabilities

In [5]:
target_seq, dict_edit_freq, dict_edit_rate, dict_coedit_freq, dict_coedit_rate = prepare_probs('TGTGTCGTTTTCGGTTGTCTG_outcomes.tsv')

##### Single Probability of position 6 and position 12 (in Python index, they are 5 and 11): 

In [6]:
print('The probability of position 6 edited: ' + str(dict_edit_rate[(5, 'edited')]))
print('The probability of position 12 edited: ' + str(dict_edit_rate[(11, 'edited')]))

The probability of position 6 edited: 0.12637847375385972
The probability of position 12 edited: 0.11424790471989413


##### Conditional probability of position 6 and position 12

In [7]:
print('The probability of position 6 edited under the condition that position 12 is edited: ' + str(dict_coedit_rate[(5, 11), ('edited', 'edited')]))
print('The probability of position 12 edited under the condition that position 6 is edited: ' + str(dict_coedit_rate[(11, 5), ('edited', 'edited')]))

The probability of position 6 edited under the condition that position 12 is edited: 0.8734729493891797
The probability of position 12 edited under the condition that position 6 is edited: 0.9662162162162162


##### The 4 probabilities above are enough to calculate the bystanders' probabilities using formula V1.
##### But for formula V2, we still need the following four probabilities:

In [8]:
print('The probability of position 6 NOT edited: ' + str(dict_edit_rate[(5, 'not-edited')]))
print('The probability of position 12 NOT edited: ' + str(dict_edit_rate[(11, 'not-edited')]))
print('The probability of position 6 edited under the condition that position 12 is NOT edited: ' + str(dict_coedit_rate[(5, 11), ('edited', 'not-edited')]))
print('The probability of position 12 edited under the condition that position 6 is NOT edited: ' + str(dict_coedit_rate[(11, 5), ('edited', 'not-edited')]))
print('The probability of position 6 NOT edited under the condition that position 12 is edited: ' + str(dict_coedit_rate[(5, 11), ('not-edited', 'edited')]))
print('The probability of position 12 NOT edited under the condition that position 6 is edited: ' + str(dict_coedit_rate[(11, 5), ('not-edited', 'edited')]))

The probability of position 6 NOT edited: 0.8736215262461403
The probability of position 12 NOT edited: 0.8857520952801059
The probability of position 6 edited under the condition that position 12 is NOT edited: 0.12652705061082026
The probability of position 12 edited under the condition that position 6 is NOT edited: 0.033783783783783786
The probability of position 6 NOT edited under the condition that position 12 is edited: 0.004418076243372886
The probability of position 12 NOT edited under the condition that position 6 is edited: 0.018052788844621515


### Caculate the probabilities of all bystanders using formula V1

In [9]:
dict_bystanders_prob = calc_prob_bystanders_v1(target_seq, 4, 16, dict_edit_rate, dict_coedit_rate)
dict_bystanders_prob

{'TGTGTCGTTTTTGGTTGTCTG': 0.11424790471989413,
 'TGTGTTGTTTTCGGTTGTCTG': 0.12637847375385972,
 'TGTGTTGTTTTTGGTTGTCTG': 0.11038817820908689}

### Caculate the probabilities of all bystanders using formula V2

In [10]:
dict_bystanders_prob = calc_prob_bystanders_v2(target_seq, 4, 16, dict_edit_rate, dict_coedit_rate)
dict_bystanders_prob

{'TGTGTCGTTTTTGGTTGTCTG': 0.0038597265108072346,
 'TGTGTTGTTTTCGGTTGTCTG': 0.01599029554477283,
 'TGTGTTGTTTTTGGTTGTCTG': 0.11038817820908689}